In [1]:
import pandas as pd
import requests
import datetime

import env

In [2]:
search_base_url = 'https://www.googleapis.com/youtube/v3/search'
videos_base_url = 'https://www.googleapis.com/youtube/v3/videos'

search_params = {
  'key': env.token,
  'part': 'snippet',
  'order': 'viewCount',
  'regionCode': 'KR',
  'type': 'video',
  'maxResults': 20,
  'fields': 'items(id/videoId, snippet(publishedAt,title))'
}
videos_params = {
  'key': env.token,
  'part': 'statistics',
  'fields': 'items/statistics'
}

search_keywords = ['영화', '명장면', '리뷰', '해석', '의미']
columns = ['영상 id', '영화 제목', '검색 키워드', '영상 제목', '조회수', '좋아요수', '댓글 수', '생성된 날짜']

tokenLimit = len(env.token)-1

In [3]:
movieInfo = pd.read_csv('movieInfo.csv').loc[:,['영화명', '개봉일']]
movieNames = list(movieInfo.loc[:, '영화명'])

In [4]:
startIndex = 65
endIndex = len(movieNames)

def get_video_list():
  columns = ['영상 id', '영화 제목', '검색 키워드', '영상 제목', '조회수', '좋아요수', '댓글 수', '생성된 날짜']
  video_list = pd.DataFrame(columns=columns)
  tokenNum = 0

  for movieName in movieNames[startIndex:endIndex]:
    print(movieName, len(video_list))
    for keyword in search_keywords:
      # parameter 설정
      publishedAfter = list(movieInfo[movieInfo['영화명'] == movieName].loc[:, '개봉일'])[0]
      year, month, day = publishedAfter.split('-')
      publishedBefore = (datetime.datetime(int(year), int(month), int(day)) + datetime.timedelta(days=30)).strftime('%Y-%m-%d')
      q = ' '.join([movieName, keyword])

      search_params['q'] = q
      search_params['publishedAfter'] = publishedAfter+'T00:00:00Z'
      search_params['publishedBefore'] = publishedBefore+'T00:00:00Z'

      res_search = requests.get(search_base_url, search_params).json()

      # 네트워크 오류 발생 시
      while 'error' in res_search.keys():
        print('[ERROR] search requests ', res_search)
        tokenNum += 1
        if tokenNum > tokenLimit: 
          print('[Error] token 오늘 할당량 끝났다~~', movieName, keyword)
          return video_list

        print('[HTTPError] token 교체')
        search_params['key'] = env.token[tokenNum]
        res_search = requests.get(search_base_url, search_params).json()

      # 응답 데이터 가공
      if 'items' in res_search.keys():
        for item in res_search['items']:
          result_video_ids = list(video_list.loc[video_list['영화 제목'] == movieName,'영상 id'])

          if item['id']['videoId'] not in result_video_ids:
            videos_params['id'] = item['id']['videoId']
            res_videos = requests.get(videos_base_url, videos_params).json()

            # 네트워크 오류 발생 시
            while 'error' in res_videos.keys():
              print('[ERROR] video requests ', res_videos)
              tokenNum += 1
              if tokenNum > tokenLimit: 
                print('[Error] token 오늘 할당량 끝났다~~', movieName)
                return video_list

              print('[HTTPError] token 교체')
              videos_params['key'] = env.token[tokenNum]
              res_videos = requests.get(videos_base_url, videos_params).json()

            if 'items' in res_videos.keys():
              try:
                video_statistics = res_videos['items'][0]['statistics']
                item_data = [[
                  item['id']['videoId'], 
                  movieName, 
                  q, 
                  item['snippet']['title'], 
                  int(video_statistics['viewCount']) if 'viewCount' in video_statistics else 0,
                  int(video_statistics['likeCount']) if 'likeCount' in video_statistics else 0,
                  int(video_statistics['commentCount']) if 'commentCount' in video_statistics else 0,
                  item['snippet']['publishedAt'], 
                ]]
                video_list = video_list.append(pd.DataFrame(item_data, columns=columns),ignore_index=True)
              except Exception as e:
                print('[ERROR] ', e, movieName, res_videos)
                return video_list


  return video_list

In [5]:
video_list = get_video_list()

미드웨이 0
시동 32
지푸라기라도 잡고 싶은 짐승들 70
미스터 주: 사라진 VIP 113
인비저블맨 140
나쁜 녀석들: 포에버 171
국제수사 197
침입자 219
스타워즈: 라이즈 오브 스카이워커 246
스파이 지니어스 276
이웃사촌 299
온워드: 단 하루의 기적 323
소리도 없이 348
버즈 오브 프레이(할리 퀸의 황홀한 해방) 384
원더 우먼 1984 424
겨울왕국 2 455
오! 문희 519
그린랜드 539
위대한 쇼맨 559
[ERROR] search requests  {'error': {'code': 403, 'message': 'The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.', 'errors': [{'message': 'The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.', 'domain': 'youtube.quota', 'reason': 'quotaExceeded'}]}}
[HTTPError] token 교체
런 590
뮬란 627
내가 죽던 날 655
[ERROR] video requests  {'error': {'code': 403, 'message': 'The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.', 'errors': [{'message': 'The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a

In [6]:
result = video_list.set_index('영상 id')

In [7]:
result.to_csv(
  ''.join(['./videoList/movieVideoList_', str(startIndex), '_', str(endIndex-1),'.csv']), 
  encoding='utf-8-sig')

In [8]:
top = lambda x: x.sort_values(by='조회수', ascending=False)[:10]
filtered_result = result.groupby('영화 제목').apply(top)
filtered_result

영화 제목        검색 키워드  \
영화 제목     영상 id                                  
검객        D3oecFZmPnM         검객         검객 영화   
          mQxVJZvflBs         검객         검객 영화   
          3pFteU56o5w         검객         검객 영화   
          oE9arnb-krE         검객         검객 영화   
          dBU9XjCn1_0         검객         검객 영화   
...                          ...           ...   
프리즌 이스케이프 xt5v39dwmYQ  프리즌 이스케이프  프리즌 이스케이프 영화   
          p0bcqMnEwUM  프리즌 이스케이프  프리즌 이스케이프 영화   
          6RRUJ8XjE4E  프리즌 이스케이프  프리즌 이스케이프 영화   
          JTWRNu_fUjE  프리즌 이스케이프  프리즌 이스케이프 영화   
          TGQPx2lL9uY  프리즌 이스케이프  프리즌 이스케이프 영화   

                                                                   영상 제목  \
영화 제목     영상 id                                                            
검객        D3oecFZmPnM     역대 최고 중 하나가 된 무협 영화, 진시황의 앞에 섰던 검객의 이야기 [영화리뷰]   
          mQxVJZvflBs       맹인 지팡이 속 숨겨진 칼날의 비밀, 검객 자토이치의 정체 [영화리뷰/결말포함]   
          3pFteU56o5w      아이를 업은 채 적들을 쓸어버리는 몰락한 가문 최강의 검객! [영화리뷰/결말포함]   
          oE9arnb-krE    신민아와 장혁이 각각 다른 영화로 돌아왔다: 디바 &amp; 검객 미리 보고 온 후기   
          dBU9XjCn1_0  검객 THE SWORDSMAN on＜KBS2 &#39;영화가좋다 I love mov...   
...                                                                  ...   
프리즌 이스케이프 xt5v39dwmYQ  [영화랑] 70년 남아공 실화 탈옥 스릴러 &#39;프리즌 이스케이프&#39; 역주...   
          p0bcqMnEwUM                       [프리즌 이스케이프] 다니엘 래드클리프 인터뷰 영상   
          6RRUJ8XjE4E                           [특별 예고편] 프리즌 이스케이프 #3 탈출   
          JTWRNu_fUjE                 열쇠로만 스릴을 유도하는 쾌감실화! / 프리즌 이스케이프 리뷰   
          TGQPx2lL9uY       [씬의 한수]코로나19로 답답하다면…영화 속 모험에 동행하라 | 뉴스A LIVE   

                          조회수  좋아요수  댓글 수                생성된 날짜  
영화 제목     영상 id                                                  
검객        D3oecFZmPnM  850379  5064   744  2020-09-25T07:21:37Z  
          mQxVJZvflBs  684936  5002  1025  2020-10-03T07:51:07Z  
          3pFteU56o5w  306156  2776   241  2020-10-11T12:06:41Z  
          oE9arnb-krE   32575   599   104  2020-09-23T08:00:12Z  
          dBU9XjCn1_0   27614   149    22  2020-10-03T02:33:33Z  
...                       ...   ...   ...                   ...  
프리즌 이스케이프 xt5v39dwmYQ    1429    13     0  2020-05-21T01:23:44Z  
          p0bcqMnEwUM    1316    19     1  2020-05-15T02:43:07Z  
          6RRUJ8XjE4E     864     4     0  2020-05-08T09:01:56Z  
          JTWRNu_fUjE     775     8     8  2020-05-08T10:39:33Z  
          TGQPx2lL9uY     756     7     0  2020-05-22T03:24:56Z  

[300 rows x 7 columns]

In [9]:
filtered_result.to_csv(
  ''.join(['./filteredVideoList/movieVideoList_',str(startIndex),'_',str(endIndex-1),'.csv']), 
  encoding='utf-8-sig'
)

### 파일 이어 붙이기

In [10]:
import glob
import pandas as pd

In [19]:
filenames = glob.glob('./filteredVideoList/*.csv')
print(filenames)

df_list = (pd.read_csv(f) for f in filenames)
concated_list = pd.concat(df_list, ignore_index=True)
concated_list = concated_list.drop(columns=['영화 제목.1'], axis=1)

['./filteredVideoList/movieVideoList_12_25.csv', './filteredVideoList/movieVideoList_65_94.csv', './filteredVideoList/movieVideoList_0_11.csv', './filteredVideoList/movieVideoList_26_64.csv']


In [20]:
concated_list.to_csv('./videoList_final.csv', encoding='utf-8-sig')